In [23]:
import pandas as pd

In [24]:
data_path = "../conversations/dsat/voip-0f41c16f2f-20130325_192310.txt"

In [25]:
with open (data_path) as f:
    data = f.read()

In [26]:
print(data)

[SYSTEM] Hello , welcome to the Cambridge restaurant system? You can ask for restaurants by area , price range or food type . How may I help you?
[USER]   i'm looking for a restaurant that serves german food
[SYSTEM] You are looking for a german restaurant right?
[USER]   no
[SYSTEM] Let me confirm , You are looking for a restaurant in the moderate price range right?
[USER]   yes
[SYSTEM] I'm sorry but there is no restaurant serving german food
[USER]   how about free
[SYSTEM] I'm sorry but there is no restaurant serving german food
[USER]   a restaurant serving french food
[SYSTEM] You are looking for a german restaurant right?
[USER]   thank you goodbye



#### Repetition of the chatbot

In [116]:
def number_repetition(file):
    with open (file) as f:
        lines = f.readlines()
        new_list =[]
        for line in lines:
            if "SYSTEM" in line:
                new_list.append(line.strip())
    #create dictionary
    new_dict = {}
    for li in new_list:
        if li in new_dict:
            new_dict[li] += 1
        else:
            new_dict[li] = 1
    num_rep = sum([value for value in new_dict.values() if value > 1])
    
    #create a matrix to calculate repetition
    m = [[0]* len(new_list) for i in range(len(new_list))]
    for i, x in enumerate(new_list):
        for j, y in enumerate(new_list):
            if x == y:
                m[i][j] = 1 
    #count number of repetition
    all_value = 0
    for li in m:
        a = sum(li)
        all_value+=a

    num_rep_exp = (all_value-len(new_list))/2
    num_rep_per = num_rep/len(lines)
    len_conversation = len(lines)
    return num_rep, num_rep_per,num_rep_exp,len_conversation

In [123]:
import glob
text_files_dsat = glob.glob("../conversations/dsat/*.txt")
text_files_sat = glob.glob("../conversations/sat/*.txt")

In [124]:
columns = ['num_rep','num_rep_per','num_rep_exp','len_conversation', 'Is_satisfied']
df_rep_dsat = pd.DataFrame(index=range(len(text_files_dsat)), columns=columns)

In [125]:
for idx,file in enumerate(text_files_dsat):
    num_rep, num_rep_per,num_rep_exp,len_conversation = number_repetition(file)
    df_rep_dsat.iloc[idx] = pd.Series({'num_rep':num_rep, 'num_rep_per':num_rep_per,
                                  'num_rep_exp':num_rep_exp,'len_conversation':len_conversation, 'Is_satisfied':0})

In [126]:
columns = ['num_rep','num_rep_per','num_rep_exp','len_conversation', 'Is_satisfied']
df_rep_sat = pd.DataFrame(index=range(len(text_files_sat)), columns=columns)

In [127]:
for idx,file in enumerate(text_files_sat):
    num_rep, num_rep_per,num_rep_exp,len_conversation = number_repetition(file)
    df_rep_sat.iloc[idx] = pd.Series({'num_rep':num_rep, 'num_rep_per':num_rep_per,
                                  'num_rep_exp':num_rep_exp,'len_conversation':len_conversation, 'Is_satisfied':1})

In [128]:
df_rep_dsat.head()

,num_rep,num_rep_per,num_rep_exp,len_conversation,Is_satisfied
0,2,0.0714286,1,28,0
1,11,0.275,19,40,0
2,5,0.3125,4,16,0
3,0,0,0,6,0
4,8,0.285714,16,28,0


In [129]:
df_rep_sat.head()

,num_rep,num_rep_per,num_rep_exp,len_conversation,Is_satisfied
0,8,0.285714,28,28,1
1,0,0,0,16,1
2,0,0,0,10,1
3,18,0.36,32,50,1
4,0,0,0,12,1


In [130]:
print('Number of disatisfied files: {}'.format(len(df_rep_dsat)))
print('Number of satisfied files: {}'.format(len(df_rep_sat)))

Number of disatisfied files: 157
Number of satisfied files: 314


In [135]:
new_df = pd.concat([df_rep_dsat,df_rep_sat])

In [137]:
len(new_df)

471

In [140]:
df_shuffle = new_df.sample(frac=1).reset_index(drop=True)

In [145]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [144]:
X = df_shuffle.iloc[:,:-1]
y = df_shuffle.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [153]:
y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [154]:
clf = LogisticRegression(random_state=0, solver='lbfgs',
                          multi_class='multinomial').fit(X_train, y_train)

In [155]:
classifier = LogisticRegression(random_state=0)
classifier.fit(X_train, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [156]:
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[23 22]
 [ 9 64]]


In [157]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(classifier.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.74
